In [15]:
## Goal: Create, clean, & prepare data for Adventure Works dataset (Supervised ML)

# Import Python pkgs pandas, numpy, matplotlib.pyplot, & seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr
import math
from sklearn import preprocessing

%matplotlib inline  
# Start of magic command which configures execution environment, to display graphics w/in notebook

In [4]:
# Load dataset, display shape, & explore first 10 rows of Pandas data frame

AW_Custs_Test = pd.read_csv('AW_testAge.csv', header=0)
print(AW_Custs_Test.shape)
AW_Custs_Test.head(10)

(500, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,Bachelors,Management,F,S,0,2,0,5,86931,52
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,Bachelors,Skilled Manual,M,M,1,2,2,4,100125,33
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,Graduate Degree,Management,F,M,1,2,0,4,103985,63
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161,39
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,High School,Manual,F,M,1,1,2,2,21876,32
5,16375,NaN,Gabriella,NaN,Hernandez,NaN,5689 Almondtree Circle,NaN,Spring Valley,California,...,Partial College,Clerical,F,M,1,1,0,1,44467,40
6,16109,NaN,Adam,K,Turner,NaN,9800 American Beauty Dr.,NaN,N. Vancouver,British Columbia,...,Bachelors,Skilled Manual,M,S,1,2,2,4,77702,33
7,15606,NaN,Marco,C,Prasad,NaN,8523 Rose Drive,C,Cranbourne,Victoria,...,Bachelors,Professional,M,M,1,3,4,4,99418,33
8,20888,NaN,Clayton,P,Raje,NaN,"4, place du Tertre",NaN,Tremblay-en-France,Seine Saint Denis,...,Partial High School,Manual,M,S,1,1,4,4,13522,31
9,20716,NaN,Kristina,V,Arun,NaN,Waldstr 29,NaN,Hannover,Nordrhein-Westfalen,...,Bachelors,Skilled Manual,F,M,1,1,1,1,46264,24


In [6]:
#Above- The new column addition now gives us a total of 26 columns.

#Below- Review data types for each column

AW_Custs_Test.dtypes

CustomerID               int64
Title                   object
FirstName               object
MiddleName              object
LastName                object
Suffix                  object
AddressLine1            object
AddressLine2            object
City                    object
StateProvinceName       object
CountryRegionName       object
PostalCode              object
PhoneNumber             object
BirthDate               object
Education               object
Occupation              object
Gender                  object
MaritalStatus           object
HomeOwnerFlag            int64
NumberCarsOwned          int64
NumberChildrenAtHome     int64
TotalChildren            int64
YearlyIncome             int64
Age                      int64
dtype: object

In [7]:
# Find missing values w/in dataset
 # Can be coded as character string, numeric value like -999 or missing value NaN, or a NULL value.
# Then determine how to treat missing values.
 # Remove rows w/ missing values
 # Remove features w/ missing values
 # Impute by replacing values w/ mean or median using simple algorithms, or more complex ones like SMOTE or expectation maximization (EM).
 # Use nearest neighbor, averaging, forward filling or backward filling.

# Shows which features are missing values

#(AW_Custs.astype(np.object) == '?').any()  # This will only work w/ strings
pd.isna(AW_Custs_Test).any()

CustomerID              False
Title                    True
FirstName               False
MiddleName               True
LastName                False
Suffix                   True
AddressLine1            False
AddressLine2             True
City                    False
StateProvinceName       False
CountryRegionName       False
PostalCode              False
PhoneNumber             False
BirthDate               False
Education               False
Occupation              False
Gender                  False
MaritalStatus           False
HomeOwnerFlag           False
NumberCarsOwned         False
NumberChildrenAtHome    False
TotalChildren           False
YearlyIncome            False
Age                     False
dtype: bool

In [8]:
#Above- Columns for Title, MiddleName, Suffix, & AddressLine2 all contain missing values in their columns.

#Below- Find out how many values are missing w/in each feature.

for col in AW_Custs_Test.columns:
    if AW_Custs_Test[col].dtype == object:
        count = 0
        count = [count + 1 for x in AW_Custs_Test[col] if pd.isna(x)]
        print(col + ' ' + str(sum(count)))

Title 496
FirstName 0
MiddleName 216
LastName 0
Suffix 499
AddressLine1 0
AddressLine2 487
City 0
StateProvinceName 0
CountryRegionName 0
PostalCode 0
PhoneNumber 0
BirthDate 0
Education 0
Occupation 0
Gender 0
MaritalStatus 0


In [9]:
#Above- Title, Suffix, & AddressLine2 are missing significant # of values, while MiddleName has missing values, it is less than the other 3 columns.

#Below- Examine data for duplicates by checking for unique IDs.

print(AW_Custs_Test.shape)
print(AW_Custs_Test.CustomerID.unique().shape)

(500, 24)
(500,)


In [10]:
# Perform some cleaning by replacing missing values
#Below- Since so many values are missing for columns Title, MiddleName, Suffix, & AddressLine2, the whole feature/columns will be removed.
   # Duplicate rows (115 rows) will be removed & will retain most recent account ID.

def clean_AW_data(AW_Custs_Test):
    
    #Remove(Drop) whole column due to many values being missing
    AW_Custs_Test.drop(['Title', 'MiddleName', 'Suffix', 'AddressLine2'], axis = 1, inplace = True)
    
    #Remove(Drop) duplicate rows, keeping last unique customer ID
    AW_Custs_Test.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
        
    return AW_Custs_Test
AW_Custs = clean_AW_data(AW_Custs_Test)

print(AW_Custs_Test.shape)
print(AW_Custs_Test.CustomerID.unique().shape)
print(AW_Custs_Test.columns)

(500, 20)
(500,)
Index(['CustomerID', 'FirstName', 'LastName', 'AddressLine1', 'City',
       'StateProvinceName', 'CountryRegionName', 'PostalCode', 'PhoneNumber',
       'BirthDate', 'Education', 'Occupation', 'Gender', 'MaritalStatus',
       'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'Age'],
      dtype='object')


In [11]:
#Above- 4 columns & 0 rows were removed from the dataset. Now 20 columns and 500 rows.

#Below- Now that duplicates & unuseful features have been removed, CustomerID is not a needed feature, so will remove this feature also.

AW_Custs_Test.drop(['CustomerID'], axis = 1, inplace = True)
print(AW_Custs_Test.shape)
AW_Custs_Test.head()

(500, 19)


,FirstName,LastName,AddressLine1,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,Courtney,Baker,8727 Buena Vista Ave.,Fremont,California,United States,94536,133-555-0128,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931,52
1,Adam,Allen,3491 Cook Street,Haney,British Columbia,Canada,V2W 1W2,252-555-0173,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125,33
2,Bonnie,Raji,359 Pleasant Hill Rd,Burbank,California,United States,91502,409-555-0193,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985,63
3,Julio,Alonso,8945 Euclid Ave.,Burlingame,California,United States,94010,175-555-0196,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161,39
4,Christy,Andersen,"42, boulevard Tremblay",Dunkerque,Nord,France,59140,1 (11) 500 555-0122,3/19/1965,High School,Manual,F,M,1,1,2,2,21876,32


In [12]:
# Compute & display summary statistics for numeric columns

AW_Custs_Test.describe()

,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.666000,1.592000,1.150000,2.188000,81795.474000,35.468000
std,0.472112,1.136715,1.596433,1.696589,40627.641775,10.974235
min,0.000000,0.000000,0.000000,0.000000,10588.000000,17.000000
25%,0.000000,1.000000,0.000000,1.000000,50273.750000,27.000000
50%,1.000000,2.000000,0.000000,2.000000,82298.000000,34.000000
75%,1.000000,2.000000,2.000000,4.000000,107067.750000,43.000000
max,1.000000,4.000000,5.000000,5.000000,184897.000000,68.000000


In [26]:
#Above- CustomerID column/feature was also removed, now for a total of 5 columns & 0 rows were removed from the dataset. We now have 500 rows and 19 columns remaining in dataset.

#Below- Save this data as a csv

AW_Custs_Test.to_csv('AW_testAge_Preped.csv', index = False, header = True)